# Revista de Educação e Pesquisa em Contabilidade – REPEC

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(url_search):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find_all('h3', {'class': 'title'})
    a_links = []
    for item in results:
        try:
            remover_links = 'editorial' in item.find('a').text.lower()
            remover_links = remover_links or 'folha de rosto' in item.find('a').text.lower()
            remover_links = remover_links or 'dados da edição ' in item.find('a').text.lower()
            if not remover_links:
                a_links.append(item.find('a')['href'])
        except:
            pass
    return a_links

In [4]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(url_search)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links

In [5]:
url_search_text = 'https://www.repec.org.br/repec/search/index?query=__query__&searchJournal=1&authors=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&searchPage=__pg_num__#results'

links = listar_links(url_search_text, 'contabilidade+ambiental', pg_num=1)

link = links[1]

link


'https://www.repec.org.br/repec/article/view/2382'

In [6]:
links

['https://www.repec.org.br/repec/article/view/1928',
 'https://www.repec.org.br/repec/article/view/2382']

In [7]:
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [8]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [9]:
def limpar_afiliacao(afiliacao):
    afiliacao = ' '.join(afiliacao.split())
    afiliacao = afiliacao.split(' - ')[0]
    afiliacao = afiliacao.split('(')[0]
    afiliacao = afiliacao.split('/')[0]
    return afiliacao.strip()

In [10]:
def captura_titulo(soup):
    try:
        title = soup.find('h1', {'class': 'page_title'}).text
        title = ' '.join(title.split())
    except:
        title = ''
    return {'Título': title}

In [11]:
captura_titulo(soup)

{'Título': 'INFLUÊNCIA DO DESEMPENHO ACADÊMICO NA PERCEPÇÃO DE JUSTIÇA NO AMBIENTE DE APRENDIZAGEM'}

In [12]:
def captura_data(soup):
    try:
        date_tag = soup.find('div', {'class': 'item published'})
        date = date_tag.find('div', {'class': 'value'}).text
        date = ' '.join(date.split())
    except:
        date = ''
    return {'Data de Publicação': date[:10]}

In [13]:
captura_data(soup)

{'Data de Publicação': '2019-12-20'}

In [14]:
def captura_resumo(soup):
    try:
        abstract_section = soup.find('section', {'class': 'item abstract'})
        abstract = abstract_section.text
        abstract = abstract.replace('Resumo:', '')
        abstract = abstract.replace('Resumo', '').strip()
        #abstract = ' '.join(abstract.split())
    except:
        abstract = ''
    return {'Resumo': abstract}

In [15]:
captura_resumo(soup)

{'Resumo': 'Objetivo: Buscar evidências que permitam identificar a relação existente entre o desempenho acadêmico de alunos de graduação em ciências contábeis e sua percepção de justiça acadêmica, segregadas nas dimensões distributiva, procedimental e interacional.\nMétodo: Consiste em uma abordagem qualitativa e quantitativa, que teve como amostra 534 alunos de graduação de Ciências Contábeis de nove universidades federais mineiras. Para abordagem quantitativa foram aplicados questionários que buscaram captar a percepção de justiça em suas três dimensões (variáveis dependentes), o desempenho acadêmico (variável explicativa) e oito variáveis de controle. Para abordagem qualitativa foram realizadas entrevistas.\nResultados: As análises quantitativas indicam que das nove variáveis testadas, apenas três variáveis se mostraram estatisticamente significativa: a variável gênero para a justiça procedimental, a variável turno para justiças procedimental e interacional e a variável reprovação p

In [16]:
def captura_palavras_chave(soup):
    try:
        kw_section = soup.find('section', {'class': 'item keywords'})
        kw = kw_section.text.replace('Palavras-chave:', '')
        kw = ' '.join(kw.split())
        kw = kw.replace(',', '.').replace(';', '.')
    except:
        kw = ''
    if kw == '':
        try:
            kw_section = soup.find('section', {'class': 'item abstract'})
            p_list = kw_section.find_all('p')
            for p in p_list:
                if 'Palavras-chave' in p.text:
                    kw_full = p.text
                    kw = kw_full.replace('Palavras-chave', '').replace(':', '')
                    kw = ' '.join(kw.split())
                    kw = kw.replace(',', '.').replace(';', '.')
        except:
            pass
    return {'Palavras-chave': kw}

In [17]:
captura_palavras_chave(soup)

{'Palavras-chave': 'Justiça no Ambiente de Aprendizagem.. Ciências Contábeis.. Universidades Federais.'}

In [18]:
def captura_autores(soup):
    autores = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        autores = authors_ul.find_all('span', {'class': 'name'})
        autores = [' '.join(a.text.split()) for a in autores]
    except:
        pass
    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [19]:
captura_autores(soup)

{'Qtd. de Autores': 4,
 'Autor 1': 'Karla Luisa Costa Sabino',
 'Autor 2': 'Jacqueline Veneroso Alves da Cunha',
 'Autor 3': 'Romualdo Douglas Colauto',
 'Autor 4': 'José Roberto de Souza Francisco',
 'Autor 5': '',
 'Autor 6': ''}

In [20]:
def captura_afiliacao(soup):
    afiliacao = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        afiliacao = authors_ul.find_all('span', {'class': 'affiliation'})
        afiliacao = [' '.join(a.text.split()) for a in afiliacao]
        afiliacao = [limpar_afiliacao(a) for a in afiliacao]
    except:
        pass
    afiliacao = completar_lista(afiliacao)
    
    key_affiliation = [f"Afiliação {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, afiliacao))

In [21]:
captura_afiliacao(soup)

{'Afiliação 1': 'Universidade Federal de Alfenas',
 'Afiliação 2': 'Universidade Federal de Minas Gerais',
 'Afiliação 3': 'Universidade Federal do Paraná',
 'Afiliação 4': 'Universidade Federal de Minas Gerais',
 'Afiliação 5': '',
 'Afiliação 6': ''}

In [22]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'Revista de Educação e Pesquisa em Contabilidade – REPEC'}
    url_search_text = 'https://www.repec.org.br/repec/search/index?query=__query__&searchJournal=1&authors=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&searchPage=__pg_num__#results'
    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'Url Artigo': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        dict_artigo = {
            **captura_data(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%Y-%m-%d')
    return df

## Pesquisa por artigos

In [23]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head(3)

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2019-06-24,FATORES CONTINGENCIAIS EXTERNOS E A MENSURAÇÃO...,Objetivo: Analisar as relações entre os fatore...,Fatores contingenciais externos. Mensuração de...,2,Alan Santos de Oliveira,Antônio André Cunha Callado,,,,,Universidade Federal do Ceará,Universidade Federal Rural de Pernambuco,,,,,https://www.repec.org.br/repec/article/view/1928,Revista de Educação e Pesquisa em Contabilidad...
1,2019-12-20,INFLUÊNCIA DO DESEMPENHO ACADÊMICO NA PERCEPÇÃ...,Objetivo: Buscar evidências que permitam ident...,Justiça no Ambiente de Aprendizagem.. Ciências...,4,Karla Luisa Costa Sabino,Jacqueline Veneroso Alves da Cunha,Romualdo Douglas Colauto,José Roberto de Souza Francisco,,,Universidade Federal de Alfenas,Universidade Federal de Minas Gerais,Universidade Federal do Paraná,Universidade Federal de Minas Gerais,,,https://www.repec.org.br/repec/article/view/2382,Revista de Educação e Pesquisa em Contabilidad...
2,2021-07-01,INSERÇÃO DO TEMA SUSTENTABILIDADE NO CURSO DE ...,Objetivo: compreender o processo de institucio...,Teoria Institucional. Sustentabilidade. Ensino...,3,Karina Rocha Henriques Gehlen,Luciano Gomes dos Reis,Kelli Juliane Favato,,,,Faculdade Positivo,Universidade Estadual de Londrina,Universidade Federal do Paraná,,,,https://www.repec.org.br/repec/article/view/2666,Revista de Educação e Pesquisa em Contabilidad...


In [24]:
df.to_csv('../data/REPEC.csv', index=False)

In [25]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2019-06-24,FATORES CONTINGENCIAIS EXTERNOS E A MENSURAÇÃO...,Objetivo: Analisar as relações entre os fatore...,Fatores contingenciais externos. Mensuração de...,2,Alan Santos de Oliveira,Antônio André Cunha Callado,,,,,Universidade Federal do Ceará,Universidade Federal Rural de Pernambuco,,,,,https://www.repec.org.br/repec/article/view/1928,Revista de Educação e Pesquisa em Contabilidad...
1,2019-12-20,INFLUÊNCIA DO DESEMPENHO ACADÊMICO NA PERCEPÇÃ...,Objetivo: Buscar evidências que permitam ident...,Justiça no Ambiente de Aprendizagem.. Ciências...,4,Karla Luisa Costa Sabino,Jacqueline Veneroso Alves da Cunha,Romualdo Douglas Colauto,José Roberto de Souza Francisco,,,Universidade Federal de Alfenas,Universidade Federal de Minas Gerais,Universidade Federal do Paraná,Universidade Federal de Minas Gerais,,,https://www.repec.org.br/repec/article/view/2382,Revista de Educação e Pesquisa em Contabilidad...
2,2021-07-01,INSERÇÃO DO TEMA SUSTENTABILIDADE NO CURSO DE ...,Objetivo: compreender o processo de institucio...,Teoria Institucional. Sustentabilidade. Ensino...,3,Karina Rocha Henriques Gehlen,Luciano Gomes dos Reis,Kelli Juliane Favato,,,,Faculdade Positivo,Universidade Estadual de Londrina,Universidade Federal do Paraná,,,,https://www.repec.org.br/repec/article/view/2666,Revista de Educação e Pesquisa em Contabilidad...
